In [1]:
from spru_package.ckks_x import CKKS_x
from spru_package.ckks_in_sagemath.ckks_package.poly import Poly

In [4]:
N = 2**13
q = 2**29
p = 2**22
delta = 2**32

C = 2**4  # Number of coefficients
h = 2**6
g = 1
L = ceil(log(C // 2, 2) / g) + log(h, 2) + 2

CKKS_x.config(N, L, q, p, delta)
CKKS_x.key_gen(h)
CKKS_x.config_SPRU(CKKS_x.sk, C, g)

In [3]:
m = Poly([randint(-p, p) if i % (N // C) == 0 else 0 for i in range(N)], N)
ct = CKKS_x.enc_poly_with_pk(m, CKKS_x.pk) % q
ct_boot = ct.SPRU()
print(ct.get_precision(ct_boot, CKKS_x.sk, C // 2))

11.67


In [7]:
ct.trace(N // 2, h * C)

A CKKS ciphertext for bootstrapping with degree N = 2^13 and modulus q = (2^51) * (2^32)^13 (level 13 out of 14)

In [ ]:
from sage.all import log, randint
import numpy as np
from spru_package.ckks_in_sagemath.ckks_package.ckks import CKKS
from spru_package.ckks_in_sagemath.ckks_package.poly import Poly
from spru_package.ckks_in_sagemath.ckks_package.fast_dft import get_grouped_F
from spru_package.ext_bit_rev import ext_bit_rev_vector

In [4]:
# Basic parameters

N = 2**13
q = 2**29
p = 2**22
delta = 2**32

n = 2**4  # Number of coefficients
h = 2**6
log_radix = 1
L = ceil(log(n // 2, 2) / log_radix) + log(h, 2) + 2

# Secret key generation

B = N // h
sk_coeffs = np.zeros(N, dtype=int)
sk_coeffs[0] = 1
for b_coeffs in range(1, h):
    j = randint(0, B - 1)
    sk_coeffs[b_coeffs * B + j] = 1
sk = Poly(sk_coeffs, N)

# CKKS configuration

CKKS.config(N, N // 2, L, q, p, delta, print_messages=True)
CKKS.key_gen(sk=sk)

The CKKS configuration is done!


In [5]:
# swk generation

for i in range(0, log(N, 2)):
    # Rotation by powers of two
    k = 5 ** (2**i)
    swk = CKKS.get_galois_swk(k, sk)
swk = CKKS.get_galois_swk(-1, sk)  # Conjugation

# Bootstrapping keys

s_tilde_list = []
cs_list = []
for u in range(2 * n):
    s_tilde = np.zeros(N // 2, dtype=np.complex128)
    for k in range(B // (2 * n)):
        for b in range(h):
            for a in range(n):
                s_tilde[k * h * n + b * n + a] = sk_coeffs[
                    b * B + u * B // (2 * n) + k
                ]
    s_tilde = ext_bit_rev_vector(s_tilde, n // 2)
    s_tilde_list.append(s_tilde)
    cs_list.append(
        CKKS.enc_poly_with_sk(
            CKKS.encode(s_tilde, is_boot=True), sk, is_boot=True
        )
    )

# Configure SlotToCoeff

eta = np.zeros(n, np.complex128)
for i in range(n):
    eta[i] = 1 if i < n // 2 else 1j
enc_eta = CKKS.encode(eta, is_boot=True)

grouped_F = get_grouped_F(n // 2, log_radix, False).copy()

grouped_poly_F = [CKKS.get_poly_matrix(A, is_boot=True) for A in grouped_F]

rotation_indices = []
for poly_matrix in grouped_poly_F:
    rotation_indices += CKKS.get_BSGS_rotation_indices(poly_matrix)
for k in rotation_indices:
    CKKS.get_galois_swk(5**k, sk)

In [6]:
# Ciphertext creation

m = Poly([randint(-p, p) if i % (N // n) == 0 else 0 for i in range(N)], N)
ct = CKKS.enc_poly_with_pk(m, CKKS.pk) % q

In [8]:
# Coefficient encodings

E_list = []
inner_factor = (q / (4 * delta * np.pi)) ** (1 / h)
outer_factor = 2 * np.pi * 1j / q
a_coeffs = ct.a.coeffs
b_coeffs = ct.b.coeffs
for u in range(2 * n):
    e = np.zeros(N // 2, dtype=np.complex128)
    for k in range(B // (2 * n)):
        for b in range(h):
            for a in range(n):
                i = b * B + u * B // (2 * n) + k
                if N // n * a >= i:
                    e_coeff = a_coeffs[N // n * a - i]
                else:
                    e_coeff = -a_coeffs[N + N // n * a - i]
                if i == 0:
                    e_coeff += b_coeffs[N // n * a]
                e[k * h * n + b * n + a] = int(e_coeff)

    e = np.exp(e * outer_factor) * inner_factor
    e = ext_bit_rev_vector(e, n // 2)
    E = CKKS.encode(e, is_boot=True)
    E_list.append(E)

In [9]:
# Bootstrapping without SlotToCoeff

out = CKKS.enc_poly_without_error(0, is_boot=True)
for u in range(2 * n):
    out += cs_list[u] @ E_list[u]
out = CKKS.trace(out, N // 2, h * n)
out = CKKS.product(out, h * n, n)
out = out - out.conjugate()
out = -(1j * out)

In [10]:
# SlotToCoeff

out = enc_eta @ out
out = out.trace(n, n // 2)
for A in grouped_poly_F:
    out = out.BSGS_left_mult(A)
out = out.boot_to_nonboot()

In [11]:
print(out.dec_to_poly(sk))
print(m)
print(out.l)
print(out.get_precision(ct, sk, n // 2))

1765760 - 9X - 22X^3 + 30X^4 + 26X^5 + 33X^6 - 4X^7 + 12X^8 + 14X^9 - 10X^10 + 48X^11 - 8X^12 + 19X^13 - 19X^14 + 2X^15 + 15X^16 - X^17 - 16X^18 + 7X^19 + 6X^20 - 16X^22 - 22X^23 - 3X^24 + 46X^25 + 25X^26 + 47X^27 + 17X^28 - 23X^29 - 10X^30 - 12X^31 + 11X^32 + 13X^33 + X^34 + 39X^35 - 7X^36 + 22X^37 - 15X^39 + 7X^40 + 8X^41 + 7X^42 - 6X^43 - 24X^44 + 7X^45 - 29X^46 + 36X^47 - 5X^48 - 18X^49 - 13X^50 - 2X^51 + 29X^52 - 38X^53 - 9X^54 - 8X^55 - 10X^56 - 21X^57 + 9X^58 - 36X^59 - 22X^60 + 15X^61 + 12X^62 - 18X^63 + 43X^64 - 18X^65 - 2X^66 + 11X^67 - 14X^68 - 8X^69 + 21X^70 + 21X^71 + 6X^72 + 48X^73 + 9X^74 - 6X^75 + 28X^76 + 12X^77 - 10X^78 + 13X^79 + 50X^80 + 3X^81 - X^82 + 28X^83 - 27X^84 + 40X^85 + 25X^86 - 26X^87 - 15X^88 - 29X^89 - 2X^90 - 21X^91 + 11X^92 - 8X^93 + 37X^94 + 14X^95 - 8X^96 - 28X^97 + 15X^98 - 15X^99 + 22X^100 + 13X^101 + 9X^102 - 35X^103 + 34X^104 - 17X^105 - 6X^106 - 38X^107 + 31X^108 + 14X^109 + X^110 - 43X^111 + 10X^112 + 4X^113 + 17X^114 - 25X^115 - X^116 + 3X^117